In [1]:
import os

In [2]:
%pwd

'D:\\Python\\MLProjectsPW\\EEG_EYE\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\Python\\MLProjectsPW\\EEG_EYE'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    data_path:Path

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [12]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.io.arff import loadarff

In [15]:
class DataTransformation:
    def __init__(self, config = DataTransformationConfig):
        self.config = config

    ## Note: You can add different data transformation techniques such as scaler , PCA and all
    # You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train test split because the data is already cleaned up


    def train_test_splitting(self):
        data = pd.DataFrame(loadarff(self.config.data_path)[0])
        data['eyeDetection'] = data['eyeDetection'].astype('int8')

        # Split the data into train and test sets. (0.75 , 0.25) split
        train , test = train_test_split(data,random_state=0)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info(f"Splited data into training and testing sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2023-08-29 23:20:57,026: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-29 23:20:57,034: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-29 23:20:57,039: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-29 23:20:57,041: INFO: common: created directory at :artifacts]
[2023-08-29 23:20:57,041: INFO: common: created directory at :artifacts/data_transformation]
[2023-08-29 23:20:57,773: INFO: 3470231904: Splited data into training and testing sets]
[2023-08-29 23:20:57,773: INFO: 3470231904: (11235, 15)]
[2023-08-29 23:20:57,773: INFO: 3470231904: (3745, 15)]
(11235, 15)
(3745, 15)
